<a href="https://colab.research.google.com/github/christyesmee/Thesis/blob/main/DataDownload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data downloading

The Data Downloading process exists of two steps:
1. Downloading Sentinel-2A imagery
2. Downloading ontology features

## Webpages

The following web pages can be visited to gain a better understanding of the basic theory on the satellite bands and using Google Earth Engine.

* [Copernicus band description](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED#bands)

* [EE Run Status](https://code.earthengine.google.com/tasks)

## Installing dependencies

In [ ]:
%%capture
!pip install SPARQLWrapper tqdm gdal
!pip install black[jupyter] --quiet

In [ ]:
import ee
import pickle
import math
import time
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
from tqdm.notebook import trange

## Defining paths

In [ ]:
from google.colab import drive

drive_path = "/content/drive"
drive.mount(drive_path)

Mounted at /content/drive


In [ ]:
# File path setup
img_dir = "17_sat_tif"  # The folder where you want to save the .tif files of the satelite images in
folder_path = f"{drive_path}/MyDrive/{img_dir}"

# Image path & scale for Sentinel-2A download
file_name = "galicia_split_image"  # Name of the .tif file
file_path = f"{folder_path}/{file_name}"
scale = 10  # pixels per meter

# Feature ontology path
ont_dir = "ont_pickles"
folder_path_ont = f"{drive_path}/MyDrive/{ont_dir}"
file_name_ont = "ont_features"
file_path_ont = f"{folder_path_ont}/{file_name_ont}"

# 1. Downloading Sentinel-2A imagery

### Google Earth Engine
The first step is to create an account in Google Cloud and create a project within an organization.

In [ ]:
# Trigger the authentication flow
ee.Authenticate()

# Initialize the library
ee.Initialize(project="christyesmee")  # this is the projectid

### Defining Satellite, Bands and AOI

In the step below you can change the bands you want to download for the image. If you want to get other bands then the ones used below, you can check out this webpage to see what bands are available to download for the Sentinel-2A imagery:  https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED#bands

In [ ]:
# Select bands (RGB and NIR) and calculate NDVI
def preprocess(image):
    # Normalize bands and calculate NDVI
    ndvi = image.normalizedDifference(["B8", "B4"]).rename("NDVI")

    # Select and rename bands
    bands = image.select(["B4", "B3", "B2", "B8"], ["Red", "Green", "Blue", "NIR"])

    # Stack bands and NDVI, cast all to Float32
    bands = bands.addBands(ndvi).toFloat()

    return bands

In [ ]:
# function calculation is WRONG, Galicia has total surface of 29574 km^2
# function calculates: Area: 186.97km²
def get_area_roots(aoi):
    aoi_area = aoi.area(maxError=1)
    total_meters = aoi_area.getInfo()
    m_root = math.sqrt(total_meters)
    km_root = m_root / 1_000
    print(f"--> Area: {round(km_root, 2):_}km²")
    return m_root, km_root

Some info to remember about latitudes and longitudes:


Latitude vs Longitude

< West to East > == Latitude (from "latus", side)

 -90 to 90 (0 over greenwitch)

< North to South > == Longitude (from "longus", length)

 90 to -90 (0 at equator)



What is used for Galicia is this:

GALICIA

Latitude: -8.599948 to -6.711535

Longitude: 41.797418 to 43.308103

<top-right>               <top-left>

43.308103, -8.599948)    (43.308103, -6.711535)

41.797418, -8.599948)    (41.797418, -6.711535)

<bot-right> <bot-left>


In [ ]:
# Name of satellite
sat_name = "COPERNICUS/S2_HARMONIZED"
cloud_filter = ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20)  # Less than 20% cloud cover

# Define the Area of Interest as a rectangle around Galicia, Spain
top_left, top_right = (43.308103, -8.599948), (43.308103, -6.711535)
bot_left, bot_right = (41.797418, -8.599948), (41.797418, -6.711535)
aoi = ee.Geometry.Rectangle([top_left[0], top_left[1], bot_right[0], bot_right[1]])
get_area_roots(aoi)

# Extract data
date_0 = "2017-01-01"
date_1 = "2017-12-30"  # PRED -> "2019-6-22"

# Split image into slices -- have to split cuz we can only download until a certain amount of pixels per batch
slices = 7
factor = (bot_right[0] - top_left[0]) / slices
x_coords = [bot_right[1] + (factor * i) for i in range(slices + 1)]
y_coords = [top_left[0] + (factor * i) for i in range(slices + 1)]
print(f"--> {x_coords} ({len(x_coords)}), delta: {x_coords[-1]-x_coords[0]}")
print(f"--> {y_coords} ({len(y_coords)}), delta: {y_coords[-1]-y_coords[0]}")
print(f"--> top-left: {x_coords[0]}, {y_coords[-1]}")
print(f"--> bot-right: {x_coords[-1]}, {y_coords[0]}")

--> Area: 186.97km²
--> [-6.711535, -6.927347142857143, -7.143159285714286, -7.358971428571429, -7.574783571428572, -7.790595714285716, -8.00640785714286, -8.222220000000002] (8), delta: -1.5106850000000023
--> [43.308103, 43.09229085714286, 42.87647871428572, 42.66066657142857, 42.44485442857143, 42.229042285714286, 42.01323014285715, 41.797418] (8), delta: -1.5106850000000023
--> top-left: -6.711535, 41.797418
--> bot-right: -8.222220000000002, 43.308103


### Downloading and splitting images

Before executing the following, go to this webpage to start the google task manager and follow the progress of downloading the images from google ee: https://code.earthengine.google.com/tasks

It may take a while before the images will show up in your folder. You can keep track of this progress in the task manager.

In [ ]:
# Image splitting
print("\n ------------------------------------------------- \n")
info_dict = {}
tasks = []
for idx in range(len(x_coords) - 1):
    (
        x_start_i,
        x_end_i,
    ) = (
        x_coords[idx],
        x_coords[idx + 1],
    )
    print(f"--> {x_start_i}, {x_end_i}")
    for jdx in range(len(x_coords) - 1):
        y_start_i, y_end_i = y_coords[jdx], y_coords[jdx + 1]
        print(f"--> {y_start_i}, {y_end_i}")

        split_name = f"{file_name}_{idx}_{jdx}"
        print(f"--> {split_name}")

        # Get geom
        aoi_temp = ee.Geometry.Rectangle([x_start_i, y_start_i, x_end_i, y_end_i])
        get_area_roots(aoi_temp)
        info_dict[(idx, jdx)] = ((x_start_i, y_start_i), (x_end_i, y_end_i))

        # Get data
        landsat = (
            ee.ImageCollection(sat_name)
            .select(["B4", "B3", "B2", "B8"])  # R G B NIR
            .filterBounds(aoi_temp)
            .filterDate(date_0, date_1)
            .filter(cloud_filter)
        )

        # Make necessary edits to data
        preprocessed = landsat.map(preprocess)

        # Prepare for image output
        image = preprocessed.mean()
        image = image.clip(aoi_temp)

        task = ee.batch.Export.image.toDrive(
            image=image,
            description=split_name,
            fileNamePrefix=split_name,
            folder=img_dir,
            scale=scale,
            region=aoi_temp,
            fileFormat="GeoTIFF",
        )

        print(f"Export task started... ")
        task.start()
        tasks.append(task)
        print("\n")


 ------------------------------------------------- 

--> -6.711535, -6.927347142857143
--> 43.308103, 43.09229085714286
--> galicia_split_image_0_0
--> Area: 20.49km²
Export task started... 


--> 43.09229085714286, 42.87647871428572
--> galicia_split_image_0_1
--> Area: 20.52km²
Export task started... 


--> 42.87647871428572, 42.66066657142857
--> galicia_split_image_0_2
--> Area: 20.56km²
Export task started... 


--> 42.66066657142857, 42.44485442857143
--> galicia_split_image_0_3
--> Area: 20.6km²
Export task started... 


--> 42.44485442857143, 42.229042285714286
--> galicia_split_image_0_4
--> Area: 20.63km²
Export task started... 


--> 42.229042285714286, 42.01323014285715
--> galicia_split_image_0_5
--> Area: 20.67km²
Export task started... 


--> 42.01323014285715, 41.797418
--> galicia_split_image_0_6
--> Area: 20.7km²
Export task started... 


--> -6.927347142857143, -7.143159285714286
--> 43.308103, 43.09229085714286
--> galicia_split_image_1_0
--> Area: 20.49km²
Export 

### Saving metadata

Now we need to save the metadata of the tif images in a pickle format to be able to access the info easily.

**IMPORTANT!!! **

Only execute when the all tasks in the task manager are fully executed.

In [22]:
# Save image info
file_path_info = f"{file_path}.pickle"
with open(file_path_info, "wb") as f:
    pickle.dump(info_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

# Load image info
file_path_info = f"{file_path}.pickle"
with open(file_path_info, "rb") as f:
    test = pickle.load(f)
print(test)

{(0, 0): ((-6.711535, 43.308103), (-6.927347142857143, 43.09229085714286)), (0, 1): ((-6.711535, 43.09229085714286), (-6.927347142857143, 42.87647871428572)), (0, 2): ((-6.711535, 42.87647871428572), (-6.927347142857143, 42.66066657142857)), (0, 3): ((-6.711535, 42.66066657142857), (-6.927347142857143, 42.44485442857143)), (0, 4): ((-6.711535, 42.44485442857143), (-6.927347142857143, 42.229042285714286)), (0, 5): ((-6.711535, 42.229042285714286), (-6.927347142857143, 42.01323014285715)), (0, 6): ((-6.711535, 42.01323014285715), (-6.927347142857143, 41.797418)), (1, 0): ((-6.927347142857143, 43.308103), (-7.143159285714286, 43.09229085714286)), (1, 1): ((-6.927347142857143, 43.09229085714286), (-7.143159285714286, 42.87647871428572)), (1, 2): ((-6.927347142857143, 42.87647871428572), (-7.143159285714286, 42.66066657142857)), (1, 3): ((-6.927347142857143, 42.66066657142857), (-7.143159285714286, 42.44485442857143)), (1, 4): ((-6.927347142857143, 42.44485442857143), (-7.143159285714286, 4

# 2. Downloading Ontology Features

### Normalising functions

These functions are to normalise the values of the features so it can be processed to torch tensors in the data processing stage.

In [ ]:
def convert_to_num(col):
    col = pd.to_numeric(col, errors="coerce")
    col = col.fillna(-1).astype(float)
    col = col.astype(float)
    return col


def normalise_column(col):
    col = (col - col.min()) / (col.max() - col.min())
    return col

### Downloading the features

The code below will download the features from the Crossforest Ontology.  

In [ ]:
for idx in trange(slices):
    for jdx in trange(slices, leave=False):
        # print(x_coords[jdx], x_coords[jdx + 1])       ## uncomment these two print statements if you want to see what coordinates for each image are used
        # print(y_coords[jdx], y_coords[jdx + 1])
        sparql = SPARQLWrapper("https://crossforest.gsic.uva.es/sparql")

        optimised_query = f"""
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
      PREFIX pos: <http://crossforest.eu/position/ontology/>
      PREFIX epsg: <http://epsg.w3id.org/ontology/axis/>
      PREFIX ifn: <https://datos.iepnb.es/def/sector-publico/medio-ambiente/ifn/>
      PREFIX taxon: <https://datos.iepnb.es/recurso/sector-publico/medio-ambiente/pliniancore/>

      SELECT ?tree ?height ?firewood ?xvalue ?yvalue ?info
      WHERE {{
        ?tree rdf:type ifn:Tree ;
              rdf:type ?info ;
              ifn:hasTotalHeightInMeters ?height ;
              ifn:hasVolumeFirewoodInM3 ?firewood ;
              pos:hasPosition ?position .

        ?position pos:hasCoordinateReferenceSystem ?refsys ;
                  pos:hasCoordinate ?coordinate ;
                  epsg:107 ?xvalue ;
                  epsg:106 ?yvalue .

        FILTER (
          ?xvalue >= {x_coords[jdx+1]} && ?xvalue <= {x_coords[jdx]} &&
          ?yvalue >= {y_coords[idx+1]} && ?yvalue <= {y_coords[idx]}
        )
      }}
      """

        # Set the query to the endpoint and set the return format to JSON
        sparql.setQuery(optimised_query)
        sparql.setReturnFormat(JSON)

        # Execute the query and convert the result to a dictionary
        results = sparql.query().convert()

        # Convert the JSON to DataFrame
        data = pd.json_normalize(results["results"]["bindings"])

        # Create a more readable DataFrame (assuming the JSON structure is typical for SPARQL results)
        data = data.applymap(lambda x: x["value"] if isinstance(x, dict) else x)

        # Clean data
        pd.set_option("display.max_colwidth", None)
        data = data.drop_duplicates(subset="tree.value", keep="first")
        data = data.drop(
            [
                "tree.type",
                "tree.value",
                "height.type",
                "height.datatype",
                "firewood.type",
                "firewood.datatype",
                "xvalue.type",
                "xvalue.datatype",
                "yvalue.type",
                "yvalue.datatype",
                "info.type",
            ],
            axis=1,
        )
        data = data.reset_index(drop=True)

        # Rename cols
        data = data.rename(
            columns={
                "xvalue.value": "xcoord",
                "yvalue.value": "ycoord",
                "height.value": "height",
                "firewood.value": "firewood",
                "info.value": "species_uri",
            }
        )

        # Delete all other rows that have other values than species
        pattern = r"^https://datos\.iepnb\.es/def/sector-publico/medio-ambiente/ifn/Species\d+$"
        data = data[data["species_uri"].str.match(pattern)]
        data = data.copy()

        # Change all strings into integers for species
        data["species_number"] = (
            data["species_uri"].str.extract(r"Species(\d+)$").astype(int)
        )
        data = data.drop(["species_uri"], axis=1)

        # Make all data numerical
        data = data.apply(convert_to_num, axis=1)

        # Normalise values between 0 and 1
        data["height"] = normalise_column(data["height"])
        data["firewood"] = normalise_column(data["firewood"])
        data["species_number"] = normalise_column(data["species_number"])

        # Save dataframe
        file_path_pandas = f"{file_path_ont}_{idx}_{jdx}.pickle"
        data.to_pickle(file_path_pandas)

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

-6.711535 -6.927347142857143
43.308103 43.09229085714286
-6.927347142857143 -7.143159285714286
43.09229085714286 42.87647871428572
-7.143159285714286 -7.358971428571429
42.87647871428572 42.66066657142857
-7.358971428571429 -7.574783571428572
42.66066657142857 42.44485442857143
-7.574783571428572 -7.790595714285716
42.44485442857143 42.229042285714286
-7.790595714285716 -8.00640785714286
42.229042285714286 42.01323014285715
-8.00640785714286 -8.222220000000002
42.01323014285715 41.797418


  0%|          | 0/7 [00:00<?, ?it/s]

-6.711535 -6.927347142857143
43.308103 43.09229085714286
-6.927347142857143 -7.143159285714286
43.09229085714286 42.87647871428572
-7.143159285714286 -7.358971428571429
42.87647871428572 42.66066657142857
-7.358971428571429 -7.574783571428572
42.66066657142857 42.44485442857143
-7.574783571428572 -7.790595714285716
42.44485442857143 42.229042285714286
-7.790595714285716 -8.00640785714286
42.229042285714286 42.01323014285715
-8.00640785714286 -8.222220000000002
42.01323014285715 41.797418


...

-6.711535 -6.927347142857143
43.308103 43.09229085714286
-6.927347142857143 -7.143159285714286
43.09229085714286 42.87647871428572
-7.143159285714286 -7.358971428571429
42.87647871428572 42.66066657142857
-7.358971428571429 -7.574783571428572
42.66066657142857 42.44485442857143
-7.574783571428572 -7.790595714285716
42.44485442857143 42.229042285714286
-7.790595714285716 -8.00640785714286
42.229042285714286 42.01323014285715
-8.00640785714286 -8.222220000000002
42.01323014285715 41.797418


  0%|          | 0/7 [00:00<?, ?it/s]

-6.711535 -6.927347142857143
43.308103 43.09229085714286
-6.927347142857143 -7.143159285714286
43.09229085714286 42.87647871428572
-7.143159285714286 -7.358971428571429
42.87647871428572 42.66066657142857
-7.358971428571429 -7.574783571428572
42.66066657142857 42.44485442857143
-7.574783571428572 -7.790595714285716
42.44485442857143 42.229042285714286
-7.790595714285716 -8.00640785714286
42.229042285714286 42.01323014285715
-8.00640785714286 -8.222220000000002
42.01323014285715 41.797418


  0%|          | 0/7 [00:00<?, ?it/s]

-6.711535 -6.927347142857143
43.308103 43.09229085714286
-6.927347142857143 -7.143159285714286
43.09229085714286 42.87647871428572
-7.143159285714286 -7.358971428571429
42.87647871428572 42.66066657142857
-7.358971428571429 -7.574783571428572
42.66066657142857 42.44485442857143
-7.574783571428572 -7.790595714285716
42.44485442857143 42.229042285714286
-7.790595714285716 -8.00640785714286
42.229042285714286 42.01323014285715
